In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings

# Carga de datos

In [2]:
df = pd.read_csv('data_output/charting_clean.csv', low_memory=False)

In [3]:
print(f'Filas y columnas: {df.shape}')
df.head()

Filas y columnas: (741257, 41)


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2nd_test,rallyCountVerification,Player_svr,Player_ret,1st_player_svr,1st_player_ret,2nd_player_svr,2nd_player_ret,1st_final,2nd_final
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0,1 (1),1,0,...,NaN,1,Novak Djokovic,Ben Shelton,6,f2n#,NaN,NaN,6 f2n#,NaN
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,1 (2),1,0,...,6 b19 f1 b2 s1 f3 f2 j2 *,8,Novak Djokovic,Ben Shelton,4n,NaN,6 f1 s1 f2,b19 b2 f3 j2*,4n,6 b19 f1 b2 s1 f3 f2 j2*
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0.0,0.0,15-15,1 (3),1,0,...,4 b28 f2 o1 *,4,Novak Djokovic,Ben Shelton,4d,NaN,4 f2,b28 o1*,4d,4 b28 f2 o1*
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0.0,0.0,15-30,1 (4),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,6 f3*,s28,NaN,NaN,6 s28 f3*,NaN
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0.0,0.0,30-30,1 (5),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,4 b3*,b37,NaN,NaN,4 b37 b3*,NaN


In [4]:
def get_matches_from_player(player,df):
    partidos = pd.Series(df['match_id'].unique())
    print(partidos.str.contains(player).sum())

# Contar tipos de golpes por jugador

In [5]:
test = df.copy()

test = test[['Player_svr', '1st_player_svr', '2nd_player_svr']] 

test['1st_player_svr'] = test['1st_player_svr'].str.split(' ')

#explode both columns
exploded = test.explode('1st_player_svr')

test2= exploded.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=True).reset_index(name='count')

In [6]:
exploded_no_serve = exploded[~exploded['1st_player_svr'].str.contains('6|4|5')]

test3= exploded_no_serve.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=True).reset_index(name='count')

In [7]:
jugadores_con_mas_cinco_partidos = df.groupby('Player_svr')['match_id'].nunique().reset_index().query('match_id > 5')

In [8]:
# Filtrar los jugadores que tengan más de 5 partidos en test2
test2 = test2[test2['Player_svr'].isin(jugadores_con_mas_cinco_partidos['Player_svr'])]
test2[test2['Player_svr']  == 'Rafael Nadal']

,Player_svr,1st_player_svr,count
51697,Rafael Nadal,f3,0.167801
51698,Rafael Nadal,4,0.112064
51699,Rafael Nadal,6,0.098068
51700,Rafael Nadal,f1,0.080778
51701,Rafael Nadal,5,0.060583
...,...,...,...
52077,Rafael Nadal,z18*,0.000016
52078,Rafael Nadal,z18w#,0.000016
52079,Rafael Nadal,z2d#,0.000016
52080,Rafael Nadal,z37w#,0.000016


In [9]:
test3[test3['Player_svr']  == 'Rafael Nadal']

,Player_svr,1st_player_svr,count
42892,Rafael Nadal,f3,0.290376
42893,Rafael Nadal,f1,0.139785
42894,Rafael Nadal,b1,0.097258
42895,Rafael Nadal,f2,0.082231
42896,Rafael Nadal,b2,0.069274
...,...,...,...
43246,Rafael Nadal,z18*,0.000027
43247,Rafael Nadal,z18w#,0.000027
43248,Rafael Nadal,z2d#,0.000027
43249,Rafael Nadal,z37w#,0.000027


# Analisis de golpes por jugador
* Por saques
* Por devoluciones
* Por golpes de punto
* Por winners
* Por errores

In [10]:
# df con
golpes = df[['Player_svr', '1st_player_svr', '2nd_player_svr']].copy()

# Now, modify the columns safely
golpes['1st_player_svr'] = golpes['1st_player_svr'].str.split(' ')
golpes_primer_pto = golpes.explode('1st_player_svr')

In [11]:
golpes['2nd_player_svr'] = golpes['2nd_player_svr'].str.split(' ')
golpes_segundo_pto = golpes.explode('2nd_player_svr')

## Saques

In [12]:
saques_continuados = golpes_primer_pto[golpes_primer_pto['1st_player_svr'].isin(['6', '4', '5'])]

saques_continuados_2do = golpes_segundo_pto[golpes_segundo_pto['2nd_player_svr'].isin(['6', '4', '5'])]


saques_errores = golpes_primer_pto[(golpes_primer_pto['1st_player_svr'].str.contains('6|5|4')) & (golpes_primer_pto['1st_player_svr'].str.len() == 2) & (~golpes_primer_pto['1st_player_svr'].str.contains(r'\*'))]
golpes_segundo_pto['2nd_player_svr'] = golpes_segundo_pto['2nd_player_svr'].fillna('')

saques_errores_2do = golpes_segundo_pto[
    (golpes_segundo_pto['2nd_player_svr'].str.contains('6|5|4')) &
    (golpes_segundo_pto['2nd_player_svr'].str.len() == 2) &
    (~golpes_segundo_pto['2nd_player_svr'].str.contains(r'\*'))
]


aces = golpes_primer_pto[(golpes_primer_pto['1st_player_svr'].str.contains('6|5|4')) & (golpes_primer_pto['1st_player_svr'].str.len() == 2) & (golpes_primer_pto['1st_player_svr'].str.contains(r'\*'))]

aces_2do = golpes_segundo_pto[(golpes_segundo_pto['2nd_player_svr'].str.contains('6|5|4')) & (golpes_segundo_pto['2nd_player_svr'].str.len() == 2) & (golpes_segundo_pto['2nd_player_svr'].str.contains(r'\*'))]

In [13]:
saques_continuados_v = saques_continuados.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=True).reset_index(name='count')

saques_final_v = saques_errores.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=True).reset_index(name='count')

aces_v = aces.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=True).reset_index(name='count')

In [14]:
get_matches_from_player('Gaston_Gaudio', df)

5


# RNN

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [16]:
"""df['all_sequences'] = df[['1st_player_svr', '1st_player_ret', '2nd_player_svr', '2nd_player_ret']].apply(lambda x: ' '.join([str(i) for i in x if pd.notna(i)]), axis=1)

# Explosión de la columna de secuencias por jugador para formar una secuencia continua de golpes
df_exploded = df[['Player_svr', 'all_sequences']].explode('all_sequences')"""

"df['all_sequences'] = df[['1st_player_svr', '1st_player_ret', '2nd_player_svr', '2nd_player_ret']].apply(lambda x: ' '.join([str(i) for i in x if pd.notna(i)]), axis=1)\n\n# Explosión de la columna de secuencias por jugador para formar una secuencia continua de golpes\ndf_exploded = df[['Player_svr', 'all_sequences']].explode('all_sequences')"

In [17]:
"""df[['Player_svr', 'Player_ret', '1st_player_svr', '1st_player_ret', '2nd_player_svr', '2nd_player_ret', 'all_sequences']].head()"""

"df[['Player_svr', 'Player_ret', '1st_player_svr', '1st_player_ret', '2nd_player_svr', '2nd_player_ret', 'all_sequences']].head()"

In [20]:
df.head()

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2nd_test,rallyCountVerification,Player_svr,Player_ret,1st_player_svr,1st_player_ret,2nd_player_svr,2nd_player_ret,1st_final,2nd_final
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0,1 (1),1,0,...,NaN,1,Novak Djokovic,Ben Shelton,6,f2n#,NaN,NaN,6 f2n#,NaN
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,1 (2),1,0,...,6 b19 f1 b2 s1 f3 f2 j2 *,8,Novak Djokovic,Ben Shelton,4n,NaN,6 f1 s1 f2,b19 b2 f3 j2*,4n,6 b19 f1 b2 s1 f3 f2 j2*
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0.0,0.0,15-15,1 (3),1,0,...,4 b28 f2 o1 *,4,Novak Djokovic,Ben Shelton,4d,NaN,4 f2,b28 o1*,4d,4 b28 f2 o1*
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0.0,0.0,15-30,1 (4),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,6 f3*,s28,NaN,NaN,6 s28 f3*,NaN
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0.0,0.0,30-30,1 (5),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,4 b3*,b37,NaN,NaN,4 b37 b3*,NaN


In [18]:
# Inicializar listas para almacenar datos procesados
players = []
sequences = []

# Iterar sobre cada fila del DataFrame original para dividir los golpes por jugador
for _, row in df.iterrows():
    # Servidor y receptor en el primer punto
    if pd.notna(row['1st_player_svr']):
        players.append(row['Player_svr'])
        sequences.append(row['1st_player_svr'])
    if pd.notna(row['1st_player_ret']):
        players.append(row['Player_ret'])
        sequences.append(row['1st_player_ret'])
    
    # Servidor y receptor en el segundo punto (si existen)
    if pd.notna(row['2nd_player_svr']):
        players.append(row['Player_svr'])
        sequences.append(row['2nd_player_svr'])
    if pd.notna(row['2nd_player_ret']):
        players.append(row['Player_ret'])
        sequences.append(row['2nd_player_ret'])

# Crear un nuevo DataFrame con los jugadores y sus respectivos golpes
df_exploded = pd.DataFrame({'Player': players, 'Sequence': sequences})

In [28]:
# Inicializar listas para almacenar datos procesados
players = []
sequences = []
match_ids = []
pts = []
set1_list = []
set2_list = []
gm1_list = []
gm2_list = []
pts_list = []  # Nueva lista para almacenar los puntajes (Pts)

# Iterar sobre cada fila del DataFrame original para dividir los golpes por jugador
for _, row in df.iterrows():
    # Servidor y receptor en el primer punto
    if pd.notna(row['1st_player_svr']):
        players.append(row['Player_svr'])
        sequences.append(row['1st_player_svr'])
        # Agregar datos adicionales
        match_ids.append(row['match_id'])
        pts.append(row['Pt'])
        set1_list.append(row['Set1'])
        set2_list.append(row['Set2'])
        gm1_list.append(row['Gm1'])
        gm2_list.append(row['Gm2'])
        pts_list.append(row['Pts'])  # Agregar puntaje

    if pd.notna(row['1st_player_ret']):
        players.append(row['Player_ret'])
        sequences.append(row['1st_player_ret'])
        # Agregar datos adicionales
        match_ids.append(row['match_id'])
        pts.append(row['Pt'])
        set1_list.append(row['Set1'])
        set2_list.append(row['Set2'])
        gm1_list.append(row['Gm1'])
        gm2_list.append(row['Gm2'])
        pts_list.append(row['Pts'])  # Agregar puntaje

    # Servidor y receptor en el segundo punto (si existen)
    if pd.notna(row['2nd_player_svr']):
        players.append(row['Player_svr'])
        sequences.append(row['2nd_player_svr'])
        # Agregar datos adicionales
        match_ids.append(row['match_id'])
        pts.append(row['Pt'])
        set1_list.append(row['Set1'])
        set2_list.append(row['Set2'])
        gm1_list.append(row['Gm1'])
        gm2_list.append(row['Gm2'])
        pts_list.append(row['Pts'])  # Agregar puntaje

    if pd.notna(row['2nd_player_ret']):
        players.append(row['Player_ret'])
        sequences.append(row['2nd_player_ret'])
        # Agregar datos adicionales
        match_ids.append(row['match_id'])
        pts.append(row['Pt'])
        set1_list.append(row['Set1'])
        set2_list.append(row['Set2'])
        gm1_list.append(row['Gm1'])
        gm2_list.append(row['Gm2'])
        pts_list.append(row['Pts'])  # Agregar puntaje

# Crear un nuevo DataFrame con los jugadores, secuencias y datos adicionales
df_exploded = pd.DataFrame({
    'Player': players,
    'Sequence': sequences,
    'match_id': match_ids,
    'Pt': pts,
    'Set1': set1_list,
    'Set2': set2_list,
    'Gm1': gm1_list,
    'Gm2': gm2_list,
    'Pts': pts_list  # Añadir la columna de puntaje
})

# Mostrar las primeras filas del DataFrame resultante
df_exploded.head()


,Player,Sequence,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts
0,Novak Djokovic,6,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0
1,Ben Shelton,f2n#,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0
2,Novak Djokovic,4n,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
3,Novak Djokovic,6 f1 s1 f2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
4,Ben Shelton,b19 b2 f3 j2*,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0


In [29]:
df_exploded

,Player,Sequence,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts
0,Novak Djokovic,6,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0
1,Ben Shelton,f2n#,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0
2,Novak Djokovic,4n,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
3,Novak Djokovic,6 f1 s1 f2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
4,Ben Shelton,b19 b2 f3 j2*,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
...,...,...,...,...,...,...,...,...,...
1679045,Nicola Pietrangeli,s38,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,173,1,1,5.0,4.0,40-AD
1679046,Luis Ayala,4 i3n#,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,174,1,1,5.0,4.0,40-40
1679047,Nicola Pietrangeli,f27,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,174,1,1,5.0,4.0,40-40
1679048,Luis Ayala,4 s3d#,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,175,1,1,5.0,4.0,40-AD


In [30]:
df_exploded_full = df_exploded.copy()

df_exploded_full.to_csv('data_output/df_exploded_full.csv', index=False)

## Con embedding

De esta manera obtenemos en dos filas la informacion de cada rally y el jugador que lo ejecuta.

In [121]:
print(df_exploded.shape)
df_exploded.head()

(1679050, 2)


,Player,Sequence
0,Novak Djokovic,6
1,Ben Shelton,f2n#
2,Novak Djokovic,4n
3,Novak Djokovic,6 f1 s1 f2
4,Ben Shelton,b19 b2 f3 j2*


In [122]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input

In [123]:
df_exploded.sample(frac=0.1, random_state=42)

,Player,Sequence
105786,Jannik Sinner,6 f3 f1
865366,Agustin Velotti,5 f3*
1303401,Lleyton Hewitt,f38 f3 f3 f1 y3n@
1321011,Carlos Moya,f27
208316,Felix Auger Aliassime,f18 b1n#
...,...,...
1610457,Henri Leconte,6d
895739,Bernard Tomic,4 b1 f3 f3 f1 f1 f1 f3 s1
1022112,Rafael Nadal,6 f3 o3
484913,Dominic Thiem,b18 s2 f1w@


In [124]:
df_exploded = df_exploded.sample(frac=0.1, random_state=42)


# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_exploded['Sequence'])
sequences = tokenizer.texts_to_sequences(df_exploded['Sequence'])
word_index = tokenizer.word_index

# Pad sequences
max_sequence_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare labels (next shot)
labels = df_exploded['Sequence'].shift(-1).fillna('').apply(lambda x: tokenizer.texts_to_sequences([x])[0] if x else [0])
labels = pad_sequences(labels, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(len(word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
"""model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.2)"""

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 9.9319e-06 - loss: 6.2758
Test Loss: 6.275824546813965
Test Accuracy: 1.2053301361447666e-05


In [125]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [126]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Uso de modelo

In [127]:
# Save the model
# Save the model using the recommended native Keras format
model.save('tennis_rnn_model_sample.keras')

# Load the model (when needed)
from tensorflow.keras.models import load_model
model = load_model('tennis_rnn_model.keras')


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [128]:
new_sequence = "4n f2 b3"  # Example sequence (shots separated by spaces)

# Tokenize the new sequence
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence])

# Pad the sequence
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length)

# Display the padded sequence
print(new_sequence_padded)


[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  7  3]]


In [129]:
import numpy as np

# Assume we have a new sequence for which we want to predict the next shot
new_sequence = "6 b19 f1 s1"

# Tokenize the new sequence
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence])

# Pad the sequence to match the input length the model expects
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length)

# Make a prediction
prediction = model.predict(new_sequence_padded)

# Get the predicted index with the highest probability
# Since the prediction output is of shape (1, sequence_length, vocabulary_size),
# we need to properly index it to extract the most probable class for the next shot
predicted_index = np.argmax(prediction[0], axis=-1)

# Extract only the next prediction (assuming you want to predict just the next shot)
# Since predicted_index can be an array, you want the last value in the sequence:
predicted_index = int(predicted_index[-1])

# Map the predicted index back to the actual shot (using the word index)
int_to_char = {idx: char for char, idx in word_index.items()}
predicted_shot = int_to_char.get(predicted_index, '')

# Display the predicted next shot
print(f"Predicted next shot: {predicted_shot}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Predicted next shot: f1


In [130]:
import numpy as np

# Assume we have a new starting sequence for which we want to predict the next shots
starting_sequence = "6 f13 f3 f3"

# Initialize the sequence with the starting point
current_sequence = starting_sequence

# Define how many additional shots we want to predict
num_shots_to_predict = 3

# Create a dictionary to map indices back to characters/shots
int_to_char = {idx: char for char, idx in word_index.items()}

for _ in range(num_shots_to_predict):
    # Tokenize the current sequence
    sequence_tokenized = tokenizer.texts_to_sequences([current_sequence])

    # Pad the sequence to match the input length that the model expects
    sequence_padded = pad_sequences(sequence_tokenized, maxlen=max_sequence_length)

    # Make a prediction
    prediction = model.predict(sequence_padded)

    # Get the predicted index with the highest probability for the next shot
    predicted_index = np.argmax(prediction[0], axis=-1)
    predicted_index = int(predicted_index[-1])  # Extract the index of the predicted shot

    # Map the predicted index back to the actual shot
    predicted_shot = int_to_char.get(predicted_index, '')

    # Append the predicted shot to the current sequence
    if predicted_shot:
        current_sequence += f" {predicted_shot}"

# Print the final sequence after all predictions
print(f"Generated sequence: {current_sequence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Generated sequence: 6 f13 f3 f3 b3 b3 b3


In [131]:
import numpy as np

# Assume we have a new sequence for which we want to predict the next shots
new_sequence = "6 f13 f3 f3"

# Tokenize the new sequence
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence])

# Pad the sequence to match the input length the model expects
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length)

# Make a prediction
prediction = model.predict(new_sequence_padded)

# Since the prediction is a 3D array, we take the first element
# which is of shape (sequence_length, vocabulary_size)
# We are interested in the last time step
prediction_last_step = prediction[0, -1]  # Get the last time step's prediction

# Get the indices of the top 3 most probable shots
top_3_indices = np.argsort(prediction_last_step)[-3:][::-1]  # Sort and take the top 3 in descending order

# Create a dictionary to map indices to shots
int_to_char = {idx: char for char, idx in word_index.items()}

# Extract the top 3 predictions and their probabilities
top_3_shots = [(int_to_char.get(idx, ''), prediction_last_step[idx]) for idx in top_3_indices]

# Display the top 3 most probable shots and their respective probabilities
for i, (shot, probability) in enumerate(top_3_shots):
    print(f"Rank {i+1}: Shot = {shot}, Probability = {probability:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Rank 1: Shot = b3, Probability = 0.2432
Rank 2: Shot = b2, Probability = 0.1994
Rank 3: Shot = s3, Probability = 0.1269


# Winner Prediction

In [132]:
# Step 1: Initialize Tokenizer and fit on all sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_exploded['Sequence'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Step 2: Tokenize sequences
tokenized_sequences = []
ending_labels = []
ending_type_labels = []

for seq in df_exploded['Sequence']:
    tokens = seq.split()
    tokenized_sequence = tokenizer.texts_to_sequences(tokens)
    tokenized_sequence = [item for sublist in tokenized_sequence for item in sublist]  # Flatten list of lists
    tokenized_sequences.append(tokenized_sequence)

    # Check if the sequence ends, and with what type
    last_token = tokens[-1]
    if last_token[-1] in ['#', '@', '*']:
        ending_labels.append(1)  # 1 means rally ends
        ending_type_labels.append({'#': 0, '@': 1, '*': 2}[last_token[-1]])  # Assign a class index to each type
    else:
        ending_labels.append(0)  # 0 means rally does not end
        ending_type_labels.append(3)  # 3 means no ending type

# Step 3: Pad sequences
max_sequence_length = max(len(seq) for seq in tokenized_sequences)
X = pad_sequences(tokenized_sequences, maxlen=max_sequence_length, padding='post')

# Convert labels to numpy arrays
y_ending = np.array(ending_labels)  # Binary labels for whether the point ends
y_ending_type = np.array(ending_type_labels)  # Multi-class labels for the type of ending

print(f"X shape: {X.shape}, y_ending shape: {y_ending.shape}, y_ending_type shape: {y_ending_type.shape}")


X shape: (167905, 42), y_ending shape: (167905,), y_ending_type shape: (167905,)


In [133]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout

# Define the input layer
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=64, input_length=max_sequence_length)(input_layer)

# LSTM layer
lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)  # Set return_sequences=False for a single output vector
dropout_layer = Dropout(0.2)(lstm_layer)

# Output for predicting if the rally ends (binary classification)
ending_output = Dense(1, activation='sigmoid', name='ending_output')(dropout_layer)

# Output for predicting the type of ending (multi-class classification)
ending_type_output = Dense(4, activation='softmax', name='ending_type_output')(dropout_layer)

# Define the model with two outputs
model = Model(inputs=input_layer, outputs=[ending_output, ending_type_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'ending_output': 'binary_crossentropy', 'ending_type_output': 'sparse_categorical_crossentropy'},
              metrics={'ending_output': 'accuracy', 'ending_type_output': 'accuracy'})


In [134]:
# Train the model with both outputs
"""history = model.fit(
    X, 
    {'ending_output': y_ending, 'ending_type_output': y_ending_type},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)"""


"history = model.fit(\n    X, \n    {'ending_output': y_ending, 'ending_type_output': y_ending_type},\n    epochs=10,\n    batch_size=32,\n    validation_split=0.2\n)"

In [135]:
# Predict for a new sequence
new_sequence = "6"  # Example sequence
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence.split()])
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(new_sequence_padded)

# Extract predictions
ending_prediction = predictions[0][0]  # Probability that the rally ends
ending_type_prediction = np.argmax(predictions[1][0])  # Predicted ending type

# Interpret the results
if ending_prediction > 0.5:
    ending_types = {0: "Forced Error (#)", 1: "Unforced Error (@)", 2: "Winner (*)"}
    print(f"Rally will end with a {ending_types.get(ending_type_prediction, 'No ending')}")
else:
    print("Rally is likely to continue.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Rally is likely to continue.


In [136]:
df_exploded

,Player,Sequence
105786,Jannik Sinner,6 f3 f1
865366,Agustin Velotti,5 f3*
1303401,Lleyton Hewitt,f38 f3 f3 f1 y3n@
1321011,Carlos Moya,f27
208316,Felix Auger Aliassime,f18 b1n#
...,...,...
1610457,Henri Leconte,6d
895739,Bernard Tomic,4 b1 f3 f3 f1 f1 f1 f3 s1
1022112,Rafael Nadal,6 f3 o3
484913,Dominic Thiem,b18 s2 f1w@


# Point ending

In [141]:
point_winning_data = df[['1stIn', '2ndIn','1st_final', '1st_final']]

In [143]:
point_winning_data.head()

,1stIn,2ndIn,1st_final,2nd_final
0,1,NaN,6 f2n#,NaN
1,0,1.0,4n,6 b19 f1 b2 s1 f3 f2 j2*
2,0,1.0,4d,4 b28 f2 o1*
3,1,NaN,6 s28 f3*,NaN
4,1,NaN,4 b37 b3*,NaN


In [145]:
#Reemplazo los valores de 1st_test por 2nd_test si 1stIn == 0, de manera que solo tengo los puntos en donde el saque entra
point_winning_data['1st_final'] = np.where(point_winning_data['1stIn'] == 0, point_winning_data['1st_final'], point_winning_data['1st_final'])
#Drop de las filas donde no entra el segundo saque
point_winning_data = point_winning_data[point_winning_data['2ndIn'] != 0]
point_winning_data.head()

C:\Users\agusd\AppData\Local\Temp\ipykernel_13648\38753786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point_winning_data['1st_final'] = np.where(point_winning_data['1stIn'] == 0, point_winning_data['1st_final'], point_winning_data['1st_final'])


,1stIn,2ndIn,1st_final,2nd_final
0,1,NaN,6 f2n#,NaN
1,0,1.0,4n,6 b19 f1 b2 s1 f3 f2 j2*
2,0,1.0,4d,4 b28 f2 o1*
3,1,NaN,6 s28 f3*,NaN
4,1,NaN,4 b37 b3*,NaN


In [155]:
point_winning_data

## Modelo point ending

In [147]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_sequences['1st_test'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Step 2: Tokenize sequences and create labels for when the point ends
tokenized_sequences = []
ending_labels = []

for seq in df_sequences['1st_test']:
    tokens = seq.split()
    
    # Generate partial sequences and labels
    for i in range(1, len(tokens)):
        partial_sequence = tokens[:i]  # Create partial sequence up to shot i
        tokenized_sequence = tokenizer.texts_to_sequences([partial_sequence])
        tokenized_sequence = [item for sublist in tokenized_sequence for item in sublist]  # Flatten list of lists
        tokenized_sequences.append(tokenized_sequence)

        # Label for whether the next shot is the end of the rally
        if tokens[i][-1] in ['#', '@', '*']:
            ending_labels.append(1)  # Next shot ends the rally
        else:
            ending_labels.append(0)  # Next shot does not end the rally

# Step 3: Pad sequences
max_sequence_length = max(len(seq) for seq in tokenized_sequences)
X = pad_sequences(tokenized_sequences, maxlen=max_sequence_length, padding='post')

# Convert labels to numpy array
y_ending = np.array(ending_labels)  # Labels for whether the next shot ends the rally

# Display data shape for verification
print(f"X shape: {X.shape}, y_ending shape: {y_ending.shape}")

X shape: (354003, 83), y_ending shape: (354003,)


In [149]:
# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=False))  # Only want the final output
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Predict the probability that the next shot ends the rally

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [150]:
# Train the model
history = model.fit(
    X, y_ending,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 156s 17ms/step - accuracy: 0.7985 - loss: 0.5052 - val_accuracy: 0.7990 - val_loss: 0.5020
Epoch 2/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 148s 17ms/step - accuracy: 0.7974 - loss: 0.5049 - val_accuracy: 0.7990 - val_loss: 0.5026
Epoch 3/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 153s 17ms/step - accuracy: 0.7981 - loss: 0.5018 - val_accuracy: 0.7996 - val_loss: 0.4902
Epoch 4/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 146s 17ms/step - accuracy: 0.7997 - loss: 0.4909 - val_accuracy: 0.8003 - val_loss: 0.4895
Epoch 5/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 148s 17ms/step - accuracy: 0.8084 - loss: 0.4848 - val_accuracy: 0.8193 - val_loss: 0.4703
Epoch 6/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 149s 17ms/step - accuracy: 0.8224 - loss: 0.4639 - val_accuracy: 0.8243 - val_loss: 0.4579
Epoch 7/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 147s 17ms/step - accuracy: 0.8249 - loss: 0.4584 - val_accuracy: 0.8242 - val_loss: 0.4602
Epoch 8/10
8851/8851 ━━━━━━━━━━━━━━━━━━━━ 149s 17ms/step - accuracy: 

In [151]:
# Save the model
model.save('tennis_rnn_when_point_ends.keras')

In [152]:
# Load the model
model_when = load_model('tennis_rnn_when_point_ends.keras')
